<a href="https://colab.research.google.com/github/gilleanr/Notion/blob/main/Actualizar_Cuentas_Notion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import requests
import json

# 🚀 API Key de Notion (⚠️ NO compartir públicamente)
NOTION_API_KEY = "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"  # Reemplázala con tu API Key

# 📌 ID de la Base de Datos de Ingresos
DATABASE_ID = "184343ed53ab80bb8b1fff6207c37c1a"  # ID de la base de ingresos

# 📌 ID de la Base de Datos de Cuentas
CUENTAS_DATABASE_ID = "18d343ed53ab8067929ec2c836a126a4"  # ID de la base de cuentas

# 🔗 URLs de la API de Notion
NOTION_URL = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
CUENTAS_URL = f"https://api.notion.com/v1/databases/{CUENTAS_DATABASE_ID}/query"

# 🏦 Encabezados para la autenticación
HEADERS = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}

def get_notion_data():
    """Obtiene todas las filas de la base de datos de Ingresos"""
    response = requests.post(NOTION_URL, headers=HEADERS)

    if response.status_code == 200:
        return response.json()
    else:
        print("❌ Error obteniendo datos de Notion:", response.text)
        return None

def get_cuenta_id(metodo_pago):
    """Busca el ID de la cuenta en la base de datos 'Cuentas' según el Método de Pago"""
    response = requests.post(CUENTAS_URL, headers=HEADERS)

    if response.status_code == 200:
        cuentas_data = response.json()

        # 🔍 Imprimir toda la respuesta JSON de Notion para depuración
        print("🔍 Respuesta completa de Notion (Cuentas):")
        print(json.dumps(cuentas_data, indent=4))

        for cuenta in cuentas_data["results"]:
            try:
                # Verificar la clave 'properties' y 'Método de Pago' en la estructura
                if "properties" in cuenta and "Método de Pago" in cuenta["properties"]:
                    metodo_pago_cuenta = cuenta["properties"]["Método de Pago"]["title"]

                    if metodo_pago_cuenta:
                        nombre_cuenta = metodo_pago_cuenta[0]["text"]["content"]
                        print(f"🆔 {cuenta['id']} - {nombre_cuenta}")  # Imprime nombres de las cuentas

                        if nombre_cuenta == metodo_pago:
                            return cuenta["id"]  # Devuelve el ID de la cuenta correcta

            except (KeyError, IndexError, TypeError) as e:
                print(f"⚠️ Error procesando la cuenta: {cuenta}. Detalles: {e}")

    print(f"⚠️ No se encontró una cuenta con el nombre '{metodo_pago}' en la tabla Cuentas.")
    return None

def update_notion_page(page_id, cuenta_id):
    """Actualiza la columna 'Cuentas' con la relación correcta"""
    update_url = f"https://api.notion.com/v1/pages/{page_id}"

    payload = {
        "properties": {
            "Cuentas": {
                "relation": [{"id": cuenta_id}]
            }
        }
    }

    response = requests.patch(update_url, headers=HEADERS, data=json.dumps(payload))

    if response.status_code == 200:
        print(f"✅ Página {page_id} actualizada con cuenta {cuenta_id}")
    else:
        print(f"❌ Error actualizando {page_id}: {response.text}")

def main():
    """Función principal que obtiene los datos y actualiza Notion"""
    data = get_notion_data()

    if not data:
        return

    for page in data["results"]:
        page_id = page["id"]
        metodo_pago = page["properties"].get("Método de Pago", {}).get("select", {}).get("name", "")

        if metodo_pago:
            cuenta_id = get_cuenta_id(metodo_pago)
            if cuenta_id:
                update_notion_page(page_id, cuenta_id)
            else:
                print(f"⚠️ No se encontró una cuenta con el nombre '{metodo_pago}' en la tabla Cuentas.")

if __name__ == "__main__":
    main()

Se han truncado las últimas 5000 líneas del flujo de salida.
                            "text": {
                                "content": "Banco de Chile",
                                "link": null
                            },
                            "annotations": {
                                "bold": false,
                                "italic": false,
                                "strikethrough": false,
                                "underline": false,
                                "code": false,
                                "color": "default"
                            },
                            "plain_text": "Banco de Chile",
                            "href": null
                        }
                    ]
                }
            },
            "url": "https://www.notion.so/Banco-de-Chile-18d343ed53ab80a8b3a7e42249495a17",
            "public_url": null
        }
    ],
    "next_cursor": null,
    "has_more": false,
    "type": "pag